# DosiPy

In [ ]:
import os
import sys

import numpy as np
try:
    import open3d as o3d
except ImportError:
    print(sys.exc_info())
from scipy import interpolate
import quadpy

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.integrate import elementwise_dblquad
from dosipy.utils.viz import (set_colorblind, scatter_2d, scatter_3d,
                              fig_config)
sys.path.append('..')
from helpers import (clean_df, export_pcd, export_fields,
                     poynting_vector, export_rect_idx)

In [ ]:
# set input data

polarization = 'te'
f = 26

if f == 26:
    edge_length = 20
else:
    edge_length = 10
area = edge_length ** 2

In [ ]:
# load surface data

if polarization == 'te':  # 6.76 w/m2
    origin = [-25.28, 17.25]
else:
    origin = [-19.77, 5.22]

df = load_ear_data(polarization, f, surface='front')
df = clean_df(df)
xyz = export_pcd(df)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()
cframe = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=9, origin=center+np.array([6, -25, -20])
)
#o3d.visualization.draw_geometries([pcd, cframe])

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_3d({'$z$ [mm]': xyz[:, 2],
                      '$x$ [mm]': xyz[:, 0],
                      '$y$ [mm]': xyz[:, 1],
                     },
                     alpha=0.0025, elev=[10], azim=[180]
                    )

In [ ]:
# postprocessing on the surface of the model

E, H = export_fields(df)
Sx, Sy, Sz = poynting_vector(E, H)

In [ ]:
set_colorblind()
label = '$|S|$ [W/m$^2$]'
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_3d({'$z$ [mm]': xyz[:, 2],
                      '$x$ [mm]': xyz[:, 0],
                      '$y$ [mm]': xyz[:, 1],
                      label: np.abs(np.sqrt(Sx ** 2 + Sy ** 2 + Sz ** 2)),
                     },
                     elev=[10], azim=[100]
                    )

In [ ]:
# rectangular area for averaging

avg_center = [origin[0]+edge_length/2,
              origin[1]+edge_length/2]
area = edge_length ** 2
pAPD_origin, idx_rect = export_rect_idx(xyz=xyz,
                                        center=avg_center,
                                        edge_length=edge_length,
                                        view='zy')
xyz_rect = xyz[idx_rect]
Sx_rect, Sy_rect, Sz_rect = Sx[idx_rect], Sy[idx_rect], Sz[idx_rect]
pcd_rect = o3d.geometry.PointCloud()
pcd_rect.points = o3d.utility.Vector3dVector(xyz_rect)
center_rect = pcd_rect.get_center()

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_3d({'$z$ [mm]': xyz_rect[:, 2],
                      '$x$ [mm]': xyz_rect[:, 0],
                      '$y$ [mm]': xyz_rect[:, 1],
                     },
                     alpha=0.1, azim=[-180]
                    )

In [ ]:
set_colorblind()
label = '$|S|$ [W/m$^2$]'
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_3d({'$z$ [mm]': xyz_rect[:, 2],
                      '$x$ [mm]': xyz_rect[:, 0],
                      '$y$ [mm]': xyz_rect[:, 1],
                      label: np.abs(np.sqrt(Sx_rect ** 2 + Sy_rect ** 2 + Sz_rect ** 2))
                     },
                     elev=[0], azim=[-180],
                    )

In [ ]:
# extract points visible from the plane wave incidence POV (-x direction)

diameter = np.linalg.norm(
    pcd_rect.get_max_bound() - pcd_rect.get_min_bound()
)
radius = 10 ** 4
camera = [center_rect[0] + diameter, center_rect[1] * 1.1, center_rect[2]]
_, mask = pcd_rect.hidden_point_removal(camera, radius)
xyz_avg = xyz_rect[mask]
Sx_avg, Sy_avg, Sz_avg = Sx_rect[mask], Sy_rect[mask], Sz_rect[mask]

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_3d({'$z$ [mm]': xyz_avg[:, 2],
                      '$x$ [mm]': xyz_avg[:, 0],
                      '$y$ [mm]': xyz_avg[:, 1],
                     },
                     alpha=0.1, azim=[-180]
                    )

In [ ]:
set_colorblind()
label = '$|S|$ [W/m$^2$]'
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_3d({'$z$ [mm]': xyz_avg[:, 2],
                      '$x$ [mm]': xyz_avg[:, 0],
                      '$y$ [mm]': xyz_avg[:, 1],
                      label: np.abs(np.sqrt(Sx_avg ** 2 + Sy_avg ** 2 + Sz_avg ** 2))
                     },
                     elev=[0], azim=[-180],
                    )

In [ ]:
# compute and orient normals

pcd_avg = o3d.geometry.PointCloud()
pcd_avg.points = o3d.utility.Vector3dVector(xyz_avg)
pcd_avg.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(33))
pcd_avg.orient_normals_consistent_tangent_plane(33)

In [ ]:
# extract normals

n_avg = np.asarray(pcd_avg.normals)

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=1.5, text_size=16, line_width=0.5)
fig, ax = scatter_3d({'$z$ [mm]': xyz_avg[::100, 2],
                      '$x$ [mm]': xyz_avg[::100, 0],
                      '$y$ [mm]': xyz_avg[::100, 1],
                     },
                     azim=[180]
                    )
ax.quiver(xyz_avg[::100, 2], xyz_avg[::100, 0], xyz_avg[::100, 1],
          n_avg[::100, 2], n_avg[::100, 0], n_avg[::100, 1], length=5, color='k');

In [ ]:
# compute APD distribution

APD_avg = np.abs(Sx_avg.real * n_avg[:, 0]
                 + Sy_avg.real * n_avg[:, 1]
                 + Sz_avg.real * n_avg[:, 2])

In [ ]:
set_colorblind()
label = '$APD$ [W/m$^2$]'
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_3d({'$z$ [mm]': xyz_avg[:, 2],
                      '$x$ [mm]': xyz_avg[:, 0],
                      '$y$ [mm]': xyz_avg[:, 1],
                      label: APD_avg,
                     },
                     elev=[0], azim=[90]
                    )

In [ ]:
# integrate

degree = 21
points = np.c_[xyz_avg[:, 2], xyz_avg[:, 1]]
bbox = [origin[0], origin[0]+edge_length, origin[1], origin[1]+edge_length]

func = interpolate.SmoothBivariateSpline(points[:, 0], points[:, 1], APD_avg, bbox=bbox, kx=3, ky=3)

scheme = quadpy.c2.get_good_scheme(degree)
val = scheme.integrate(lambda x: func(x[0], x[1], grid=False),
                       quadpy.c2.rectangle_points([origin[0], origin[0]+edge_length],
                                                  [origin[1], origin[1]+edge_length]))
APD_av = val / area
APD_av